In [1]:
from codebase.classes import Data, Particles
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp



## 2 factor Sim EZ

In [2]:
existing_directory = None
task_handle = 'sim_ez'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)




Creating new directory: ./log/20201207_230922_sim_ez/


In [3]:
# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 4, 
    size = 100,
    random_seed = 0
    )
    
exp_data.generate()

In [4]:
model_num = 1
## setup particles
param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
particles = Particles(
    name = 'normal',
    model_num = model_num,
    size = 1000,
    param_names = param_names,
    latent_names = latent_names)
particles.set_log_dir(log_dir)
if gen_model:
    particles.compile_prior_model()
    particles.compile_model()
else:
    particles.load_prior_model()
    particles.load_model()

particles.sample_prior_particles(exp_data.get_stan_data()) # sample prior particles
particles.reset_weights() # set weights to 0
log_lklhds = np.empty(exp_data.size)
degeneracy_limit = 0.5
for t in tqdm(range(exp_data.size)):
    particles.get_incremental_weights(
        exp_data.get_stan_data_at_t(t)
        )
    log_lklhds[t] =  particles.get_loglikelihood_estimate()
    particles.update_weights()
    
    if (essl(particles.weights) < degeneracy_limit * particles.size) and (t+1) < exp_data.size:
        particles.resample_particles()
        particles.jitter(exp_data.get_stan_data_upto_t(t+1))
        particles.reset_weights()
    else:
        particles.update_weights()

    save_obj(particles, 'particles', log_dir)
    

100%|██████████| 100/100 [54:03<00:00, 32.44s/it]


In [5]:

save_obj(log_lklhds, 'log_lklhds', log_dir)

print('\n\n')
marg_lklhd = np.exp(logsumexp(log_lklhds))
print('Marginal Likelihood %.5f'%marg_lklhd)

for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(particles.particles[name])
    w = exp_and_normalise(particles.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))





Marginal Likelihood 0.08616


Estimate
[-0.   -0.05  0.1   0.   -0.01 -0.  ]


Estimate
[[0.93 0.54 0.49 0.11 0.13 0.1 ]
 [0.54 1.01 0.44 0.1  0.12 0.09]
 [0.49 0.44 1.02 0.09 0.11 0.08]
 [0.11 0.1  0.09 0.91 0.44 0.33]
 [0.13 0.12 0.11 0.44 1.09 0.39]
 [0.1  0.09 0.08 0.33 0.39 0.76]]


In [7]:
marg_lklhd

0.08616251372582091